In [163]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta, date
import numpy as np
import time
import os
import asyncio, aiohttp, nest_asyncio
from aiohttp_retry import RetryClient, ExponentialRetry
nest_asyncio.apply()
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0'}

In [164]:
#https://stackoverflow.com/questions/23267409/how-to-implement-retry-mechanism-into-python-requests-library

import logging
import requests

from requests.adapters import HTTPAdapter, Retry

# logging.basicConfig(level=logging.DEBUG)

s = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[ 404, 502, 503, 504 ])
s.mount('http://', HTTPAdapter(max_retries=retries))

pwd = os.getcwd()
if 'L2_Fees' in pwd:
    prepend = ''
else:
    prepend = 'L2_Fees/'

In [165]:
trailing_num_days = 365#90

start_date = date.today()-timedelta(days=trailing_num_days +1)

start_date = max(start_date, date(2021,11,12))
print(start_date)
# start_date = datetime.strptime('2022-07-13', '%Y-%m-%d').date()

2021-11-12


In [166]:
eth_prices = r.get(
                'https://api.coingecko.com/api/v3/coins/ethereum/market_chart?vs_currency=usd&days='
                + str(trailing_num_days)
                + '&interval=daily'
                )
ethp = pd.DataFrame(eth_prices.json()['prices'])
ethp.rename(columns={0:'date',1:'eth_price'},inplace=True)
ethp['date'] = pd.to_datetime(ethp['date'], unit ='ms')
ethp['date'] = ethp['date'].dt.strftime("%Y-%m-%d")
ethp.drop_duplicates(subset=['date'],inplace=True,keep='first')

In [167]:
statuses = {x for x in range(100, 600)}
statuses.remove(200)
statuses.remove(429)

In [168]:
async def get_cryptostats_api(api_core_string,dt_string ):
        dt_string = dt_string.strftime("%Y-%m-%d")
        # print(dt_string)
        i = 0
        res = pd.DataFrame()
        # #try 5 times
        # while i < 5:
        #         try: 
        # async with aiohttp.ClientSession() as session:
        retry_client = RetryClient()
        async with retry_client.get(api_core_string + dt_string, retry_options=ExponentialRetry(attempts=10), raise_for_status=statuses) as response:
                try:
                        prot_req = await response.json()
                        res = pd.json_normalize(prot_req['data']).reset_index()
                        res['date'] = dt_string
                except Exception as e:
                        raise Exception("Could not convert json")
        await retry_client.close()
                
                # prot_req = await retry_client.get(api_core_string + dt_string, headers=header)
                # prot_req = await prot_req.json()
        
                # except:
                #         i = i+1
                #         time.sleep(1)
                #         continue
                # break
        return res

# async def gather_with_concurrency(n, *tasks):
#     semaphore = asyncio.Semaphore(n)
#     async def sem_task(task):
#         async with semaphore:
#             return await task
        

In [169]:
# ALL Relevant API Strings
# https://cryptostats.community/discover/fees
# This is dependent on crypto stats, eventually maybe we mod this to pull from TheGraph, but this is simplest por ahora
# https://docs.cryptostats.community/tutorial/how-to-query-with-rest-api/making-get-requests
        #How to make a single query
        #To make a single query on a collection. Send a GET request to
        #https://api.cryptostats.community/api/v1/<collection-id>/<query>/<arg>
        
        # How to make multiple queries
        # To make multiple queries on a collection, send a GET request to
        # https://api.cryptostats.community/api/v1/<collection-id>/<query-1>,<query-2>/<query-1-arg>,<query-2-arg>

api_core = 'https://api.cryptostats.community/api/v1/fees/oneDayTotalFees/' #Can only pull everything all at once. Odd.
date_rng = pd.date_range(start=start_date, end=date.today()-timedelta(days=1))
date_rng_str = ','.join(x.strftime("%Y-%m-%d") for x in date_rng)
# print(date_rng[0].strftime("%Y-%m-%d"))

In [170]:
def get_range(date_range):
        data_dfs = []
        fee_df = []
        # for dt in date_range:
        #         await asyncio.gather()
        #         data_dfs.append(res_df)
        #         # res.columns
        # try:
        #         loop.close()
        # except:
        #         #nothing
        loop = asyncio.get_event_loop()
        tasks = [get_cryptostats_api(api_core,dt) for dt in date_range]
        # print(tasks)
        data_dfs = loop.run_until_complete(asyncio.gather(*tasks, return_exceptions=True))
        # print(date_range)
        # loop.close()
        # print(data_dfs)
        fee_df = pd.concat(data_dfs)
        return fee_df

In [171]:


fdf = get_range(date_rng)
# r.get('https://api.cryptostats.community/api/v1/fees/oneDayTotalFees/2022-05-21')
# print(r)

In [172]:
fdf['results.oneDayTotalFees'] = fdf['results.oneDayTotalFees'].fillna(0)
display(fdf)

,index,id,bundle,results.oneDayTotalFees,metadata.source,metadata.icon,metadata.name,metadata.category,metadata.description,metadata.feeDescription,...,metadata.shortName,metadata.events,errors.oneDayTotalFees,metadata.protocolShutdown,metadata.tokenSymbol,metadata.bundle,metadata.subcategory,metadata.tokenticker,metadata.tokencoingecko,date
0,0,compound,None,729130.234330,Messari Subgraph,"data:image/svg+xml;base64,PHN2ZyB2aWV3Qm94PSIw...",Compound,lending,Compound is an open borrowing & lending protocol.,Interest fees are paid from borrowers to lenders.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-12
1,1,polymarket,None,3291.721455,The Graph Protocol,"data:image/svg+xml;base64,PHN2ZyB2aWV3Qm94PSIw...",Polymarket,dex,Polymarket is a prediction market.,Trading fees are paid by traders to liquidity ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-12
2,2,balancer-v1,balancer,20082.411754,The Graph Protocol,"data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0...",Balancer,dex,Balancer is a decentralized exchange & asset p...,Trading fees are paid by traders to liquidity ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-12
3,3,balancerv2,balancer,438497.945373,The Graph Protocol,"data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0...",Balancer,dex,Balancer is a decentralized exchange & asset p...,Trading fees are paid by traders to liquidity ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-12
4,4,balancerv2-polygon,balancer,29861.875715,The Graph Protocol,"data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0...",Balancer,dex,Balancer is a decentralized exchange & asset p...,Trading fees are paid by traders to liquidity ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-11-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,100,curve-optimism,curve,262.264780,The Graph Protocol,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Curve,dex,"Curve is a community-owned permissionless, dec...",Trading fees are paid by traders to liquidity ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-31
101,101,curve-polygon,curve,7.037025,The Graph Protocol,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Curve,dex,"Curve is a community-owned permissionless, dec...",Trading fees are paid by traders to liquidity ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-31
102,102,curve-avalanche,curve,46.920480,The Graph Protocol,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Curve,dex,"Curve is a community-owned permissionless, dec...",Trading fees are paid by traders to liquidity ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-31
103,103,curve-fantom,curve,58.497125,The Graph Protocol,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA...",Curve,dex,"Curve is a community-owned permissionless, dec...",Trading fees are paid by traders to liquidity ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-08-31


In [173]:
data_fdf = fdf[['id','metadata.blockchain','results.oneDayTotalFees','metadata.name','metadata.category','date']]
data_fdf = data_fdf.merge(ethp,on='date',how='left')

data_fdf = data_fdf.groupby(['date','metadata.blockchain','metadata.name']).sum()
data_fdf.reset_index(inplace=True)
# Exclude the chain's fees
data_fdf = data_fdf[data_fdf['metadata.blockchain'] != data_fdf['metadata.name']]

data_fdf['oneDayTotalFees_ETH'] = data_fdf['results.oneDayTotalFees']/data_fdf['eth_price']
data_fdf

,date,metadata.blockchain,metadata.name,results.oneDayTotalFees,eth_price,oneDayTotalFees_ETH
0,2021-11-12,Arbitrum One,Aave,0.000000,4732.924450,0.000000
1,2021-11-12,Arbitrum One,Abracadabra.money,7750.820530,4732.924450,1.637639
3,2021-11-12,Arbitrum One,Balancer,14351.882616,4732.924450,3.032350
4,2021-11-12,Arbitrum One,Curve,818.164268,4732.924450,0.172867
5,2021-11-12,Arbitrum One,GMX,336729.298756,4732.924450,71.146139
...,...,...,...,...,...,...
22555,2022-08-31,Polygon,Quickswap,13203.749102,1525.400116,8.655925
22556,2022-08-31,Polygon,SushiSwap,7229.561683,1525.400116,4.739453
22557,2022-08-31,Polygon,Uniswap,81240.531596,1525.400116,53.258506
22559,2022-08-31,Terra,Terraswap,0.000000,1525.400116,0.000000


In [174]:
data_fdf_op = data_fdf[data_fdf['metadata.blockchain'] == 'Optimism']

# display(data_fdf_op)

data_fdf_chain = data_fdf.groupby(['date','metadata.blockchain']).sum()
data_fdf_chain.reset_index(inplace=True)

data_fdf_chain = data_fdf_chain[data_fdf_chain['metadata.blockchain'] == 'Optimism']
data_fdf_chain.reset_index(inplace=True,drop=True)

data_fdf_chain.sort_values(by =['date','results.oneDayTotalFees'], ascending = [True,False],inplace = True)
data_fdf_chain['fees_30_day_avg'] = data_fdf_chain.groupby('metadata.blockchain')['results.oneDayTotalFees'].rolling(30,min_periods=1).mean().reset_index(0,drop=True)
data_fdf_chain['fees_90_day_avg'] = data_fdf_chain.groupby('metadata.blockchain')['results.oneDayTotalFees'].rolling(90,min_periods=1).mean().reset_index(0,drop=True)

data_fdf_chain['fees_30_day_avg_eth'] = data_fdf_chain.groupby('metadata.blockchain')['oneDayTotalFees_ETH'].rolling(30,min_periods=1).mean().reset_index(0,drop=True)
data_fdf_chain['fees_90_day_avg_eth'] = data_fdf_chain.groupby('metadata.blockchain')['oneDayTotalFees_ETH'].rolling(90,min_periods=1).mean().reset_index(0,drop=True)


#compound monthly growth rates
# https://velawoodlaw.com/glossary-term/compounded-monthly-growth-rate-cmgr/#:~:text=Compounded%20Monthly%20Growth%20Rate%20(CMGR)%20is%20a%20calculation%20that%20helps,of%20Months)%20-1%5D.
# CMGR = (Latest Month/ First Month)^(1/# of Months) -1].
data_fdf_chain['fees_1q_cmgr'] = (
                                data_fdf_chain['fees_30_day_avg']
                                / data_fdf_chain.groupby('metadata.blockchain')['fees_30_day_avg'].shift(30*3*1)
                                ) ** (1/(3*1)) - 1

data_fdf_chain['fees_3q_cmgr'] = (
                                data_fdf_chain['fees_30_day_avg']
                                / data_fdf_chain.groupby('metadata.blockchain')['fees_30_day_avg'].shift(30*3*3)
                                ) ** (1/(3*3)) - 1
data_fdf_chain['fees_1q_cmgr_eth'] = (
                                data_fdf_chain['fees_30_day_avg_eth']
                                / data_fdf_chain.groupby('metadata.blockchain')['fees_30_day_avg_eth'].shift(30*3*1)
                                ) ** (1/(3*1)) - 1

data_fdf_chain['fees_3q_cmgr_eth'] = (
                                data_fdf_chain['fees_30_day_avg_eth']
                                / data_fdf_chain.groupby('metadata.blockchain')['fees_30_day_avg_eth'].shift(30*3*3)
                                ) ** (1/(3*3)) - 1
# #if focus on op

display(data_fdf_chain)

,date,metadata.blockchain,results.oneDayTotalFees,eth_price,oneDayTotalFees_ETH,fees_30_day_avg,fees_90_day_avg,fees_30_day_avg_eth,fees_90_day_avg_eth,fees_1q_cmgr,fees_3q_cmgr,fees_1q_cmgr_eth,fees_3q_cmgr_eth
0,2021-11-12,Optimism,40815.985817,37863.395603,7.497568,40815.985817,40815.985817,7.497568,7.497568,NaN,NaN,NaN,NaN
1,2021-11-13,Optimism,49346.104574,37480.850844,9.409156,45081.045196,45081.045196,8.453362,8.453362,NaN,NaN,NaN,NaN
2,2021-11-14,Optimism,60655.396776,37331.987986,11.867277,50272.495723,50272.495723,9.591334,9.591334,NaN,NaN,NaN,NaN
3,2021-11-15,Optimism,52604.347582,37223.579154,9.760630,50855.458687,50855.458687,9.633658,9.633658,NaN,NaN,NaN,NaN
4,2021-11-16,Optimism,65334.882656,36665.691350,12.079934,53751.343481,53751.343481,10.122913,10.122913,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,2022-08-27,Optimism,189845.407368,12081.512892,113.253484,250986.269774,192870.851572,121.576921,94.295163,0.580142,0.191565,0.884388,0.316717
289,2022-08-28,Optimism,182980.083035,11968.443728,111.710524,247372.933410,194468.374386,120.686607,95.362656,0.572794,0.191301,0.873327,0.317716
290,2022-08-29,Optimism,257789.512770,11509.976403,148.527010,249751.418567,192600.278888,122.935399,94.678317,0.483892,0.193717,0.700106,0.321878
291,2022-08-30,Optimism,282284.496816,12441.419648,147.559162,251652.390553,186246.693670,124.996511,91.532808,0.343846,0.195362,0.473822,0.325603


In [175]:
fig = px.line(data_fdf_op, x="date", y="results.oneDayTotalFees", color='metadata.name', title = 'Fees Earned on Optimism (USD)')
fig.update_layout(yaxis_tickprefix = '$')
# fig.show()

fig.write_image(prepend + "img_outputs/svg/app_fees_on_op.svg") #prepend + 
fig.write_image(prepend + "img_outputs/png/app_fees_on_op.png") #prepend + 
fig.write_html(prepend + "img_outputs/app_fees_on_op.html", include_plotlyjs='cdn')

In [176]:
fig_chain = px.line(data_fdf_chain, x="date", y="results.oneDayTotalFees", color = "metadata.blockchain", title = 'Sum Fees Earned on apps by Chain (USD)')
fig_chain.update_layout(yaxis_tickprefix = '$')
# fig_chain.show()

fig_chain.write_image(prepend + "img_outputs/svg/app_fees_by_chain.svg") #prepend + 
fig_chain.write_image(prepend + "img_outputs/png/app_fees_by_chain.png") #prepend + 
fig_chain.write_html(prepend + "img_outputs/app_fees_by_chain.html", include_plotlyjs='cdn')

In [177]:
fig_chain_30d = px.line(data_fdf_chain, x="date", y="results.oneDayTotalFees", color = "metadata.blockchain", title = 'Sum Fees Earned on apps by Chain (USD)')
fig_chain_30d.update_layout(yaxis_tickprefix = '$')
fig_chain_30d.show()

In [180]:
# ! jupyter nbconvert --to python optimism_app_fees.ipynb

  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=7eb964600cf0c98ac3274482cf38cfef1f8d44969479ef7839e5a6d3097eea61
  Stored in directory: /Users/michaelsilberling/Library/Caches/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
INFO: Successfully saved requirements file in /Users/michaelsilberling/Documents/GitHub/msilb7-crypto-queries/L2_Fees/requirements.txt
